In [1]:
import matplotlib.pyplot as plt
import numpy as np
from Board import Board
from Piece import Piece
from Player import Player
from Game import Game
from shutil import copyfile
import time
import csv
import pdb
import json
from datetime import datetime
import os
import torch

In [2]:
identifier = "20241119190000"
s_model = "PTC"
import_string = 'from ' + s_model + ' import ' + s_model + ' as sm' # create self_play model import string
exec(import_string, globals())
black_model = sm("black_model", identifier)
red_model = sm("red_model", identifier)
red_player = Player(model = red_model, color = "Red") # create the red player assigning model and color
black_player = Player(model = black_model, color = "Black") # create the black player assigning model and color
game = Game(red_player = red_player, black_player = black_player, jump_rule = True, number = 0, side = "black")

cuda
Using initialized parameters.


Dice roll or maximize? R


cuda
Using initialized parameters.


Dice roll or maximize? R


In [3]:
b, b_pieces = black_model.get_input_vector(game.board, "Black", None)
r, r_pieces = black_model.get_input_vector(game.board, "Black", None)

In [4]:
np.size(b_pieces)

409

In [5]:
r.size

128

In [17]:
np.size(b_pieces)

409

In [12]:
reward = np.array([[0, 1, 0, -1]])

In [13]:
reward.shape

(1, 4)

In [14]:
reward_non_zero_mask = np.squeeze(reward, axis=0) != 0

In [15]:
reward_non_zero_mask

array([False,  True, False,  True])

In [18]:
val = np.arange(0, 4).reshape(1,4)

In [19]:
val

array([[0, 1, 2, 3]])

In [21]:
filtered_val = val[:, reward_non_zero_mask]

In [22]:
filtered_val

array([[1, 3]])

In [27]:
v = np.argmax(val)

In [28]:
print(v)

3


In [93]:
epsilon = 1e-8
X = torch.tensor([[1.5, 2, 0],[0, 2, 3],[2.5, 0, 3]])
filtered_X = torch.tensor([[1.5, 2, 0],[2.5, 0, 3]])
Y = torch.tensor([[1, 0, 0],[0, 1, 0], [0, 0, 1]])
filtered_Y = torch.tensor([[1, 0, 0], [0, 0, 1]])
log_prob = torch.log(X + epsilon)
filtered_log_prob = torch.log(filtered_X + epsilon)
reward = torch.tensor([[1], [0], [-1]])
filtered_reward = torch.tensor([[1], [-1]])
reward_win = torch.where(reward == 1, 1, 0)
reward_loss = torch.where(reward == -1, -1, 0)
filtered_reward_win = torch.where(filtered_reward == 1, 1, 0)
filtered_reward_loss = torch.where(filtered_reward == -1, -1, 0)
log_prob_move = log_prob[Y == 1].unsqueeze(1)
filtered_log_prob_move = filtered_log_prob[filtered_Y == 1].unsqueeze(1)
cost_win = - reward_win * log_prob_move
filtered_cost_win = - filtered_reward_win * filtered_log_prob_move
cost_loss = - reward_loss * log_prob_move
filtered_cost_loss = - filtered_reward_loss * filtered_log_prob_move

In [94]:
Y

tensor([[1, 0, 0],
        [0, 1, 0],
        [0, 0, 1]])

In [95]:
log_prob

tensor([[  0.4055,   0.6931, -18.4207],
        [-18.4207,   0.6931,   1.0986],
        [  0.9163, -18.4207,   1.0986]])

In [96]:
filtered_log_prob

tensor([[  0.4055,   0.6931, -18.4207],
        [  0.9163, -18.4207,   1.0986]])

In [97]:
log_prob_move

tensor([[0.4055],
        [0.6931],
        [1.0986]])

In [98]:
filtered_log_prob_move

tensor([[0.4055],
        [1.0986]])

In [99]:
cost_win

tensor([[-0.4055],
        [ 0.0000],
        [ 0.0000]])

In [100]:
filtered_cost_win

tensor([[-0.4055],
        [ 0.0000]])

In [101]:
cost_loss

tensor([[0.0000],
        [0.0000],
        [1.0986]])

In [102]:
filtered_cost_loss

tensor([[0.0000],
        [1.0986]])

In [85]:
reward_win

tensor([[1],
        [0],
        [0]])

In [103]:
cost_loss.sum()

tensor(1.0986)

In [104]:
filtered_cost_loss.sum()

tensor(1.0986)